In [13]:
import os
import sys

import pandas as pd

# Doing this to import the changed version of the library
sys.path.append('../../GoogleApiSupport')
import drive
import spreadsheets

In [2]:
TEST_WRITE_FILE_ID = '19Q8y1uR8SD27GiVN200e87ufoMklH2KBh2NfyQTEt9Q'
TEST_READONLY_FILE_ID = '1cMTfxikXMAgmdVXj3PKuD1fX_vPnQmO5teg15zGarOc'

GOOGLE_OAUTH_CREDENTIALS = r''
GOOGLE_APPLICATION_CREDENTIALS = r''

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = GOOGLE_APPLICATION_CREDENTIALS
os.environ["GOOGLE_OAUTH_CREDENTIALS"] = GOOGLE_OAUTH_CREDENTIALS

## Copy a file

You can copy an entire spreadsheet document. And set a new name.

In [3]:
# Create a copy from the TEST_READONLY_FILE_ID
my_copied_file = drive.copy_file(TEST_READONLY_FILE_ID, 'my_new_file')

Copying file 1cMTfxikXMAgmdVXj3PKuD1fX_vPnQmO5teg15zGarOc with name my_new_file


In [4]:
my_copied_file

'17E9XQyI4VM6fupmXlcO5v2apWCAwqnmcogB0fbtQXHQ'

In [5]:
spreadsheets.add_sheet(my_copied_file, 'tmp__new_page_test')
spreadsheets.add_sheet(my_copied_file, 'tmp__new_page_test_2')

{'spreadsheetId': '17E9XQyI4VM6fupmXlcO5v2apWCAwqnmcogB0fbtQXHQ',
 'replies': [{'addSheet': {'properties': {'sheetId': 447547761,
     'title': 'tmp__new_page_test_2',
     'index': 4,
     'sheetType': 'GRID',
     'gridProperties': {'rowCount': 1000, 'columnCount': 26}}}}]}

In [6]:
# Checking if we have the 2 previous created pages
response = spreadsheets.get_sheet_names(my_copied_file)
response

['hello', 'world', 'third_sheet', 'tmp__new_page_test', 'tmp__new_page_test_2']

In [7]:
# Retrieving spreadsheet sheets find sheet id's in order to 
# delete the last created sheet "tmp__new_page_test_2"
# If we want full document info we can do get_info(spreadsheet_id) instead

spreadsheets.get_sheets(my_copied_file)

[{'properties': {'sheetId': 0,
   'title': 'hello',
   'index': 0,
   'sheetType': 'GRID',
   'gridProperties': {'rowCount': 1000, 'columnCount': 26}}},
 {'properties': {'sheetId': 1641602497,
   'title': 'world',
   'index': 1,
   'sheetType': 'GRID',
   'gridProperties': {'rowCount': 1000, 'columnCount': 26}}},
 {'properties': {'sheetId': 1109152438,
   'title': 'third_sheet',
   'index': 2,
   'sheetType': 'GRID',
   'gridProperties': {'rowCount': 1000, 'columnCount': 26}}},
 {'properties': {'sheetId': 561207794,
   'title': 'tmp__new_page_test',
   'index': 3,
   'sheetType': 'GRID',
   'gridProperties': {'rowCount': 1000, 'columnCount': 26}}},
 {'properties': {'sheetId': 447547761,
   'title': 'tmp__new_page_test_2',
   'index': 4,
   'sheetType': 'GRID',
   'gridProperties': {'rowCount': 1000, 'columnCount': 26}}}]

In [9]:
spreadsheets.delete_sheet(my_copied_file, 447547761)

{'spreadsheetId': '17E9XQyI4VM6fupmXlcO5v2apWCAwqnmcogB0fbtQXHQ',
 'replies': [{}]}

In [10]:
response = spreadsheets.get_sheet_names(my_copied_file)
response

['hello', 'world', 'third_sheet', 'tmp__new_page_test']

In [11]:
# We can also access to a specific sheet and get its values
# to a dataframe object.
# By default first row will be the header in pandas.Dataframe

df_from_drive = spreadsheets.download_sheet_to_pandas(
    my_copied_file, # Id of the spreadsheet
    'hello' # Name of the sheet
)

df_from_drive.head()

,r,o,c
0,3,9,9
1,9,9,4
2,3,4,5


In [12]:
# We can request the first row in sheet to be the first row 
# of pandas content by generating a generic header. A, B, C...
# we will need to set the has_header to False

df_from_drive = spreadsheets.download_sheet_to_pandas(
    my_copied_file, # Id of the spreadsheet
    'hello', # Name of the sheet
    has_header = False
)

df_from_drive.head()

,A,B,C
0,r,o,c
1,3,9,9
2,9,9,4
3,3,4,5


In [14]:
# And we can paste our data (from a DataFrame) in a sheet

my_data = pd.DataFrame([[3,9,9],[9,9,4],[3,4,5]], columns = ['r','o','c'])

spreadsheets.pandas_to_sheet(my_copied_file, 'world', my_data)

{'spreadsheetId': '17E9XQyI4VM6fupmXlcO5v2apWCAwqnmcogB0fbtQXHQ',
 'totalUpdatedRows': 4,
 'totalUpdatedColumns': 3,
 'totalUpdatedCells': 12,
 'totalUpdatedSheets': 1,
 'responses': [{'spreadsheetId': '17E9XQyI4VM6fupmXlcO5v2apWCAwqnmcogB0fbtQXHQ',
   'updatedRange': 'world!A1:C4',
   'updatedRows': 4,
   'updatedColumns': 3,
   'updatedCells': 12}]}

In [15]:
# If we use starting_cell parameter, data will start pasting 
# at this cell
spreadsheets.pandas_to_sheet(my_copied_file, 'world', my_data, starting_cell='F8')

{'spreadsheetId': '17E9XQyI4VM6fupmXlcO5v2apWCAwqnmcogB0fbtQXHQ',
 'totalUpdatedRows': 4,
 'totalUpdatedColumns': 3,
 'totalUpdatedCells': 12,
 'totalUpdatedSheets': 1,
 'responses': [{'spreadsheetId': '17E9XQyI4VM6fupmXlcO5v2apWCAwqnmcogB0fbtQXHQ',
   'updatedRange': 'world!F8:H11',
   'updatedRows': 4,
   'updatedColumns': 3,
   'updatedCells': 12}]}

In [16]:
# And we can clear sheet content by

spreadsheets.clear_sheet(my_copied_file, 'world')

{'spreadsheetId': '17E9XQyI4VM6fupmXlcO5v2apWCAwqnmcogB0fbtQXHQ',
 'clearedRange': 'world!A1:Z1000'}